In [ ]:
## We need city csv; we need places shp file for the FIPS

In [1]:
import pandas as pd
import geopandas as gpd
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

In [11]:
# load city info used in Heat Pump Calculator (HPC)
city_url = 'https://github.com/leungkp/akwlib-export/blob/main/data/v01/city.csv' + '?raw=true'
city_HPC = pd.read_csv(city_url)

In [12]:
city_HPC.head()

,ID,Name,FIPS,TMYid,Latitude,Longitude,ERHRegionID,WAPRegionID,ImprovementCostLevel,FuelRefer,...,ElecUtilities,GasPrice,aris_city,census_city,census_area,ancsa_region,railbelt,hub,avg_elec_usage,name2match
0,1,Anchorage,203000,702730,61.151960,-149.864100,2,2,1,0,...,"[('Chugach Electric South- Residential', 1), (...",0.988132,Anchorage,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,True,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality
1,2,Adak,200065,704540,51.848896,-176.635422,2,3,5,0,...,"[('Adak -TDX Residential', 677), ('Adak - TDX ...",NaN,Adak,Adak city,Aleutians West Census Area,Aleut,Affordable Energy Strategy Area,False,"[471.936998147, 423.45531781, 460.81549341, 42...",adak city
2,3,Akutan,201090,704890,54.134945,-165.772446,2,3,5,0,...,"[('Akutan, City of - Residential', 9), ('Akuta...",NaN,Akutan,Akutan city,Aleutians East Borough,Aleut,Affordable Energy Strategy Area,False,"[500.99156233, 439.984315846, 456.510655517, 4...",akutan city
3,4,Allakaket,201860,701740,66.561386,-152.649445,4,4,5,0,...,"[('Allakaket - APT - Residential', 249), ('All...",NaN,Allakaket,Allakaket city,Yukon-Koyukuk Census Area,Doyon,Affordable Energy Strategy Area,False,"[338.509000744, 291.889556239, 276.804469551, ...",allakaket city
4,5,Angoon,203440,703710,57.496647,-134.578900,1,1,3,0,...,[('Inside Passage Electric Coop - Residential'...,NaN,Angoon,Angoon city,Hoonah-Angoon Census Area,Sealaska Corporation,Affordable Energy Strategy Area,False,"[277.080558981, 257.757954545, 237.15561277, 2...",angoon city


In [4]:
# download and unzip blocks manually because of its big size
zipurl = 'https://live.laborstats.alaska.gov/cen/maps/gis/akplaces_2010.zip'
with urlopen(zipurl) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('/tmp/Places2010')

In [5]:
places = gpd.read_file('/tmp/Places2010/')

In [6]:
## match city with places (FIPS)
city_HPC['name2match'] = city_HPC['census_city'].str.lower()
places['name2match'] = places['NAME'].str.lower()

In [7]:
city_HPC['name2match'].isin(places['name2match']).mean()

1.0

In [10]:
places2match = places[['name2match', 'FIPS']]
merged_data = pd.merge(city_HPC, places2match, on='name2match', how='left')
merged_data.head()

,ID,Name,FIPS_x,TMYid,Latitude,Longitude,ERHRegionID,WAPRegionID,ImprovementCostLevel,FuelRefer,...,GasPrice,aris_city,census_city,census_area,ancsa_region,railbelt,hub,avg_elec_usage,name2match,FIPS_y
0,1,Anchorage,203000,702730,61.151960,-149.864100,2,2,1,0,...,0.988132,Anchorage,Anchorage municipality,Anchorage municipality,Cook Inlet Regional (CIRI),Railbelt,True,"[674.3176110825, 616.54369648225, 568.80201815...",anchorage municipality,0203000
1,2,Adak,200065,704540,51.848896,-176.635422,2,3,5,0,...,NaN,Adak,Adak city,Aleutians West Census Area,Aleut,Affordable Energy Strategy Area,False,"[471.936998147, 423.45531781, 460.81549341, 42...",adak city,0200065
2,3,Akutan,201090,704890,54.134945,-165.772446,2,3,5,0,...,NaN,Akutan,Akutan city,Aleutians East Borough,Aleut,Affordable Energy Strategy Area,False,"[500.99156233, 439.984315846, 456.510655517, 4...",akutan city,0201090
3,4,Allakaket,201860,701740,66.561386,-152.649445,4,4,5,0,...,NaN,Allakaket,Allakaket city,Yukon-Koyukuk Census Area,Doyon,Affordable Energy Strategy Area,False,"[338.509000744, 291.889556239, 276.804469551, ...",allakaket city,0201860
4,5,Angoon,203440,703710,57.496647,-134.578900,1,1,3,0,...,NaN,Angoon,Angoon city,Hoonah-Angoon Census Area,Sealaska Corporation,Affordable Energy Strategy Area,False,"[277.080558981, 257.757954545, 237.15561277, 2...",angoon city,0203440


In [9]:
merged_data['FIPS'] = pd.to_numeric(merged_data['FIPS'])

KeyError: 'FIPS'

In [51]:
merged_data.columns.tolist()

['ID',
 'Name',
 'Latitude',
 'Longitude',
 'ERHRegionID',
 'WAPRegionID',
 'ImprovementCostLevel',
 'FuelRefer',
 'FuelCityID',
 'Oil1Price',
 'Oil2Price',
 'PropanePrice',
 'BirchPrice',
 'SprucePrice',
 'CoalPrice',
 'SteamPrice',
 'HotWaterPrice',
 'MunicipalSalesTax',
 'BoroughSalesTax',
 'TMYid',
 'TMYname',
 'ElecUtilities',
 'GasPrice',
 'aris_city',
 'census_city',
 'census_area',
 'ancsa_region',
 'railbelt',
 'hub',
 'avg_elec_usage',
 'name2match',
 'FIPS']

In [53]:
cols = ['ID',
 'Name',
 'FIPS',
 'TMYid',
 'Latitude',
 'Longitude',
 'ERHRegionID',
 'WAPRegionID',
 'ImprovementCostLevel',
 'FuelRefer',
 'FuelCityID',
 'Oil1Price',
 'Oil2Price',
 'PropanePrice',
 'BirchPrice',
 'SprucePrice',
 'CoalPrice',
 'SteamPrice',
 'HotWaterPrice',
 'MunicipalSalesTax',
 'BoroughSalesTax',
 'TMYname',
 'ElecUtilities',
 'GasPrice',
 'aris_city',
 'census_city',
 'census_area',
 'ancsa_region',
 'railbelt',
 'hub',
 'avg_elec_usage',
 'name2match'
 ]

In [55]:
merged_data = merged_data[cols]

In [17]:
merged_data = city_HPC
merged_data.to_csv('data/v01/city.csv', index=False)
merged_data.to_pickle('data/v01/city.pkl', compression = "bz2")

In [18]:
## amend the meta file
meta_file = pd.read_csv('data/v01/ERA5/ERA5_meta.csv')

In [19]:
merged_data2 = merged_data[['FIPS', 'TMYid']].rename(columns={'TMYid': 'tmy_id'})

In [20]:
merged_data2.head()

,FIPS,tmy_id
0,203000,702730
1,200065,704540
2,201090,704890
3,201860,701740
4,203440,703710


In [21]:
meta_file_matched = meta_file.merge(merged_data2, on='tmy_id', how='left')

In [22]:
meta_file_matched['FIPS'] = pd.to_numeric(meta_file_matched['FIPS'])

KeyError: 'FIPS'

In [26]:
meta_file_matched = meta_file
meta_file_matched.to_csv('data/v01/ERA5/ERA5_meta.csv', index=False)
meta_file_matched.to_pickle('data/v01/ERA5/ERA5_meta.pkl', compression = "bz2")